In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
hits=pd.read_csv('../input/hits.csv')
triplets=pd.read_csv('../input/triplets.csv')
len(triplets)

In [ ]:
triplets.drop(np.where(triplets['score']>0.02)[0],inplace=True)
len(triplets)

In [ ]:
inner_pairs=set()
outer_pairs=set()
for i,j,k in triplets[['inner','middle','outer']].values.astype(np.int32):
    inner_pairs.add((i,j))
    outer_pairs.add((j,k))
isolated_trips=[not ((j,k) in inner_pairs or (i,j) in outer_pairs) for i,j,k in triplets[['inner','middle','outer']].values.astype(np.int32)]

In [ ]:
triplets.drop(triplets.index[np.where(isolated_trips)[0]],inplace=True)
len(triplets)

In [ ]:
import networkx as nx
from tqdm import tqdm_notebook as tqdm
def encode_pair(a,b):
    return a*1e6+b
def decode_pair(p):
    return int(p/1e6),int(p % 1e6)
# G = nx.Graph()
graph = {}
all_pairs=set()
hit_pairs=[set() for _ in range(len(hits))]
radius=0.7
min_drho=1.0
for (i,j,k),score in zip(tqdm(triplets[['inner','middle','outer']].values.astype(np.int32)),triplets['score'].values):
    in_pair = encode_pair(i,j)
    out_pair = encode_pair(j,k)
    all_pairs.update([in_pair,out_pair])
    hit_pairs[j].update([in_pair,out_pair])
    hit_pairs[i].add(in_pair)
    hit_pairs[k].add(out_pair)
#     G.add_edge(in_pair,out_pair,weight=score)
    if in_pair in graph:
        graph[in_pair].append((out_pair,score))
    else:
        graph[in_pair]=[(out_pair,score)]
    if out_pair in graph:
        graph[out_pair].append((in_pair,score))
    else:
        graph[out_pair]=[(in_pair,score)]

In [ ]:
def flat_neighbors(pair, eps):
    return [nbr for nbr,score in graph[pair] if score < eps]
def run_dbscan(eps=0.02,min_count=1):
    pair_labels={}
    C = 0
    min_count=1
    noise = -1
    # DBSCAN Algo
    for pair in tqdm(all_pairs):
        if pair in pair_labels:
            continue
        neighbors=flat_neighbors(pair,eps)
        neighbor_set=set(neighbors)
        if len(neighbors) < min_count:
            pair_labels[pair]=noise
            continue
        C = C + 1
        pair_labels[pair] = C
        for npair in neighbors:
            if npair in pair_labels:
                if pair_labels[npair] == noise:
                    pair_labels[npair] = C
                continue
            pair_labels[npair] = C
            nneighbors = flat_neighbors(npair,eps)
            if len(nneighbors) > min_count:
                new_neighbors=set(nneighbors).difference(neighbor_set)
                neighbors.extend(list(new_neighbors))
                neighbor_set.update(new_neighbors)
    return pair_labels

In [ ]:
pair_labels=run_dbscan(0.008)

In [ ]:
def label_hits(hits,pair_labels,restrict_to_labels=None):
    hit_labels=np.zeros(len(hits),dtype=np.int64)
    for i in tqdm(range(len(hits))):
        this_hit_labels=[pair_labels[p] for p in hit_pairs[i] if pair_labels[p] > 0]
        if restrict_to_labels is not None:
            this_hit_labels=[x for x in this_hit_labels if x in restrict_to_labels]
        if len(this_hit_labels) > 0:
            labels,counts=np.unique(this_hit_labels,return_counts=True)
            hit_labels[i]=labels[np.argmax(counts)]
    return hit_labels
def get_track_labels(hits,pair_labels,min_count=4):
    hit_labels=label_hits(hits,pair_labels)
    labels,counts=np.unique(hit_labels,return_counts=True)
    restricted_labels=set(labels[np.where(counts>min_count)])
    hit_labels=label_hits(hits,pair_labels,restricted_labels)
    return hit_labels
#hits['track_id']=get_track_labels(hits,pair_labels)

In [ ]:
def full_score(hits, verbose=True, count_scoring=False, return_cluster_bins=False):
    scoring=np.count_nonzero if count_scoring else np.sum
    total_weight=0.0
    track_bins=[50,80,95,200]
    cluster_bins=[[],[],[],[]]
    overflow_bins=[]
    unmatched_bins=[]
    good_weights=np.zeros(len(track_bins))
    lost_weights=np.zeros(len(track_bins))
    remain_weights=np.zeros(len(track_bins))
    overflow_weight=0.0
    unmatched_weight=0.0
    total_weight=scoring(hits['weight'].values)
    tdf=hits.groupby('track_id')
    for group,gdf in hits.groupby('particle_id'):
        track_ids,track_counts=np.unique(gdf['track_id'].values,return_counts=True)
        good_pct=max(track_counts)/len(gdf)*100
        good_id=track_ids[np.argmax(track_counts)]
        good_weight=scoring(gdf[gdf['track_id']==good_id]['weight'].values)
        particle_weight=scoring(gdf['weight'].values)
        if good_pct>50.0:
            tbin=[i for i,b in enumerate(track_bins) if b<good_pct][-1]
            if good_id > 0:
                group_frac=max(track_counts)/len(tdf.get_group(good_id))
                if group_frac>0.5:
                    good_weights[tbin]+=good_weight
                    lost_weights[tbin]+=particle_weight-good_weight
                    cluster_bins[tbin].append(good_id)
                else:
                    overflow_weight+=particle_weight
                    overflow_bins.append(good_id)
            else:
                remain_weights[tbin]+=good_weight
                lost_weights[tbin]+=particle_weight-good_weight
        else:
            unmatched_weight+=particle_weight
            unmatched_bins.append(good_id)
    if verbose:
        print('overflow / unmatched: {:.4f} / {:.4f}'.format(overflow_weight/total_weight, unmatched_weight/total_weight))
        print('bin: good / lost / remaining')
        for i in range(len(good_weights)-1):
            print('{}%: {:.4f} / {:.4f} / {:.4f}'.format(track_bins[i],good_weights[i]/total_weight,lost_weights[i]/total_weight,remain_weights[i]/total_weight))
        print('total: {:.4f} / {:.4f} / {:.4f}'.format(np.sum(good_weights)/total_weight,np.sum(lost_weights)/total_weight,np.sum(remain_weights)/total_weight))
    if (return_cluster_bins):
        return cluster_bins,overflow_bins,unmatched_bins
    return np.sum(good_weights)/total_weight

In [ ]:
full_score(hits,count_scoring=False)

In [ ]:
# eps 0.01: 0.5325 / .1385 / .2750
# eps 0.007: 0.5675 / 0.1062 / 0.3061
# eps 0.005: 0.5404 / 0.1038 / 0.3361
# eps 0.003: 0.4613 / 0.1050 / 0.4120

In [ ]:
def get_scored_clusters(track_labels, eps):
    track_labels_order=np.argsort(track_labels)
    last_label=-1
    all_clusters=[]
    for h in tqdm(track_labels_order):
        l=track_labels[h]
        if l>0:
            if l != last_label:
                all_clusters.append([h])
                last_label=l
            else:
                all_clusters[-1].append(h)
    all_clusters=[c for c in all_clusters if len(c)<20 and len(c)>3]
    cluster_scores=[len(c)/eps for c in all_clusters]
    return all_clusters,cluster_scores

In [ ]:
all_clusters=[]
all_scores=[]
all_eps=[]
eps_list=[0.003,0.005,0.008,0.01,0.014,0.02]
for eps in eps_list:
    round_pair_labels=run_dbscan(eps)
    round_track_labels=get_track_labels(hits,round_pair_labels,min_count=3)
    round_clusters,round_scores=get_scored_clusters(round_track_labels,eps)
    all_clusters.extend(round_clusters)
    all_scores.extend(round_scores)
    all_eps.extend([eps for _ in range(len(round_clusters))])

In [ ]:
part_clusters={}
for i,c in enumerate(all_clusters):
    particle_ids,particle_counts=np.unique(hits.loc[c,'particle_id'].values,return_counts=True)
    good_pct=max(particle_counts)/len(c)*100
    good_id=particle_ids[np.argmax(particle_counts)]
    if good_pct>50.0:
        if good_id in part_clusters:
            part_clusters[good_id].append(i)
        else:
            part_clusters[good_id]=[i]

In [ ]:
all_scores=[]
for c in all_clusters:
    cdf=hits.loc[c]
    particle_ids,particle_counts=np.unique(cdf['particle_id'].values,return_counts=True)
    good_pct=max(particle_counts)/len(c)*100
    good_id=particle_ids[np.argmax(particle_counts)]
    if good_pct>50.0:
        good_weight=np.sum(cdf[cdf['particle_id']==good_id]['weight'].values)
        cluster_weight=np.sum(cdf['weight'].values)
        score=2*good_weight-cluster_weight
        all_scores.append(-score)
    else:
        all_scores.append(0)

In [ ]:
all_scores=[np.power(len(c)-12,2)*np.power(eps,.3) for c,eps in zip(all_clusters,all_eps)]

In [ ]:
all_scores=[score_cluster_linear(hits.loc[c]) for c in tqdm(all_clusters)]

In [ ]:
all_scores=[s+40/len(c) for s,c in zip(linear_scores,all_clusters)]

In [ ]:
linear_scores=all_scores

In [ ]:
all_scores=[eps if eps > .009 else 1 for eps in all_eps]

In [ ]:
cluster_score_order=np.argsort(np.array(all_scores))
hit_labels=np.zeros(len(hits),dtype=np.int64)-1
for c in cluster_score_order:
    cluster=all_clusters[c]
    unassigned_hits=[h for h in cluster if hit_labels[h]<0]
    if len(unassigned_hits)/len(cluster) < 0.5:
        continue
    for h in unassigned_hits:
        hit_labels[h]=c

In [ ]:
hits['track_id']=hit_labels
full_score(hits)

In [ ]:
bin_cs,o_cs,un_cs=full_score(hits,return_cluster_bins=True)

In [ ]:
[(x,all_scores[x]) for x in un_cs if x > 0]

In [ ]:
un_cs

In [ ]:
hits.sample(n=20)

In [ ]:
all_scores[24354]

In [ ]:
t=hits.query('track_id==14696')
score_cluster_linear(t)
t

In [ ]:
hits.loc[hits.index[all_clusters[26891]]].sort_values('rho')

In [ ]:
hits.query('particle_id==4504286822137856').sort_values('rho')

In [ ]:
hits.query('track_id==13627').sort_values('rho')

In [ ]:
potential_clusters=[(i,c) for i,c in enumerate(all_clusters) if 11440 in c]
potential_clusters

In [ ]:
[decode_pair(p) for p in hit_pairs[57199]]

In [ ]:
hits.loc[potential_clusters[0][1]]

In [ ]:
[all_scores[i] for i,c in potential_clusters]

In [ ]:
np.log(0.01)

In [ ]:
potential_clusters

In [ ]:

hits.loc[hits.index[potential_clusters[0]]]

In [ ]:
sigma_map={7: 0.03, 8: 0.03, 9: 0.03, 12: 0.3, 13: 0.3, 14: 0.3, 16: 3, 17: 3, 18: 3}
hits['sigma']=[sigma_map[x] for x in hits['volume_id'].values]
hits['r-1']=1.0/hits['r'].values

In [ ]:
i=7
chits=hits.loc[all_clusters[i]]
chits

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
X=chits[['r','rho']].values
y=chits['z'].values
lr.fit(X,y,1/chits['sigma'].values)
(lr.predict(X)-y)/chits['sigma'].values

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
def score_cluster_linear(chits):
    lr=LinearRegression()
    X=chits[['u','r-1']].values
    y=chits['v'].values
    lr.fit(X,y,1/chits['sigma'].values)
    duv=(lr.predict(X)-y)*np.power(chits['r'].values,2)/chits['sigma'].values
    lr=LinearRegression()
    X=chits[['r','rho']].values
    y=chits['z'].values
    lr.fit(X,y,1/chits['sigma'].values)
    drz=(lr.predict(X)-y)/chits['sigma'].values
    return 90/len(chits)+np.mean(np.log(1+drz*drz+duv*duv)/np.log(2))

In [ ]:
score_cluster_linear(chits)

In [ ]:
all_pairs_list=np.array(list(all_pairs))
decoded_pairs_list=np.array([list(decode_pair(p)) for p in all_pairs_list])
pair_label_list=np.array([pair_labels[p] for p in all_pairs_list])
max_pair_labels_list=np.array([max_pair_labels[l] for l in all_pairs_list])
max_label=max(pair_label_list)+1

In [ ]:
max_label

In [ ]:
pairs=pd.DataFrame()
pairs['idx1']=decoded_pairs_list[:,0]
pairs['idx2']=decoded_pairs_list[:,1]
pairs['label']=pair_label_list
pairs['region_label']=max_pair_labels_list
pairs.head(10)

In [ ]:
grouped_pairs=pairs.groupby('label')

In [ ]:
def fit_core_find_inliers(chits,lhits,deg=2,double_hit_bias=10):
    fx=np.poly1d(np.polyfit(chits['rho'].values,chits['x'].values,deg))
    fy=np.poly1d(np.polyfit(chits['rho'].values,chits['y'].values,deg))
    fz=np.poly1d(np.polyfit(chits['rho'].values,chits['z'].values,deg))
    rho=lhits['rho'].values
    xerr=(lhits['x'].values-fx(rho))
    yerr=(lhits['y'].values-fy(rho))
    zerr=(lhits['z'].values-fz(rho))
    err_tot=np.log(1+(xerr*xerr+yerr*yerr+zerr*zerr)/np.power(lhits['sigma'].values,2))/np.log(2)
    lhits['err']=err_tot
    # this section adds error to any dupe module_id
    sorted_lhits=lhits.sort_values('err')
    used_modules=set()
    for i,ids in zip(sorted_lhits.index,sorted_lhits[['volume_id','layer_id','module_id']].values):
        a,b,c=ids
        if (a,b,c) in used_modules:
            lhits.loc[i,'err']=lhits.loc[i,'err']+double_hit_bias
        used_modules.add((a,b,c))
    cum_err=np.sort(err_tot) + 50/np.array(range(1,len(err_tot)+1))
    max_idx=np.argmin(cum_err)
    inliers=lhits.sort_values('err').index[:max_idx+1]
    return lhits.loc[inliers],lhits,cum_err[max_idx]
def iterate_find_inliers(chits,lhits,max_iters=10):
    last_inliers=set(chits.index)
    for _ in range(max_iters):
        chits,lhits,score=fit_core_find_inliers(chits,lhits)
        # print('score: {}'.format(score))
        new_inliers=set(chits.index)
        if (new_inliers==last_inliers):
            break
        last_inliers = new_inliers
    return chits,lhits,score

In [ ]:
grouped_pairs.get_group(41375)

In [ ]:
keep_hit_columns=['x','y','z','rho','sigma','volume_id','layer_id','module_id','particle_id','track_id']
group=grouped_pairs.get_group(41375)
region=pairs.query('region_label==120806')
ch=np.unique(group[['idx1','idx2']].values)
rh=np.unique(region[['idx1','idx2']].values)
chits=hits.loc[ch,keep_hit_columns]
lhits=hits.loc[rh,keep_hit_columns]
chits,lhits,score=fit_core_find_inliers(chits,lhits)

In [ ]:
chits

In [ ]:
chits

In [ ]:
lhits.sort_values('err')

In [ ]:
all_groups=np.array(list(restricted_labels))
#all_groups=np.arange(1,max_label)
group_scores={}
keep_hit_columns=['x','y','z','rho','sigma','volume_id','layer_id','module_id']
#keep_hit_columns=['x','y','z','rho','sigma','volume_id','layer_id','module_id','particle_id','track_id']
for g in tqdm(all_groups):
    if g in grouped_pairs.groups:
        group=grouped_pairs.get_group(g)
        lh,lc=np.unique(group[['idx1','idx2']].values,return_counts=True)
        n_core=2+int(np.log(len(lh))/np.log(2))
        lhits=hits.loc[lh,keep_hit_columns]
        if len(lh)>n_core:
            chits=hits.loc[lh[np.argsort(lc)][::-1][:n_core],keep_hit_columns]
        else:
            chits=hits.loc[lh,keep_hit_columns]
        chits,lhits,score=iterate_find_inliers(chits,lhits)
        group_scores[g]=score

In [ ]:
good_group_list=np.array([g for g in all_groups if g in group_scores])
group_scores_list=np.array([group_scores[g] for g in good_group_list])

In [ ]:
ordered_groups=good_group_list[np.argsort(group_scores_list)]
ordered_scores=np.sort(group_scores_list)

In [ ]:
assigned_core_hits=set()
hits['best_score']=50.0
keep_hit_columns=['x','y','z','rho','sigma','volume_id','layer_id','module_id','particle_id','track_id']
for i in tqdm(range(len(ordered_groups))):
    g=ordered_groups[i]
    score=ordered_scores[i]
    group=grouped_pairs.get_group(g)
    filtered_group=[[a,b] for a,b in group[['idx1','idx2']].values if not ((a in assigned_core_hits) or (b in assigned_core_hits))]
    lh,lc=np.unique(filtered_group,return_counts=True)
    if len(lh) < 4:
        continue
    n_core=2+int(np.log(len(lh))/np.log(2))
    lhits=hits.loc[lh,keep_hit_columns]
    if len(lh)>n_core:
        chits=hits.loc[lh[np.argsort(lc)][::-1][:n_core],keep_hit_columns]
    else:
        chits=hits.loc[lh,keep_hit_columns]
    chits,lhits,new_score=iterate_find_inliers(chits,lhits)
    if (new_score > score+1):
        continue
    for core_hit in chits.index:
        hits.loc[core_hit,'track_id']=g
        hits.loc[core_hit,'best_score']=0
        assigned_core_hits.add(core_hit)
    for label_hit in lhits.index:
        label_hit_error = lhits.loc[label_hit, 'err']
        if hits.loc[label_hit,'best_score'] > label_hit_error:
            hits.loc[label_hit,'best_score'] = label_hit_error
            hits.loc[label_hit,'track_id'] = g

In [ ]:
full_score(hits)

In [ ]:
assigned_core_hits=set()
for g,score in zip(ordered_groups,ordered_scores):
    group=grouped_pairs.get_group(g)
    lh,lc=np.unique(group[['idx1','idx2']].values,return_counts=True)
    n_core=2+int(np.log(len(lh))/np.log(2))
    lhits=hits.loc[lh,keep_hit_columns]
    if len(lh)>n_core:
        chits=hits.loc[lh[np.argsort(lc)][::-1][:n_core],keep_hit_columns]
    else:
        chits=hits.loc[lh,keep_hit_columns]
    chits,lhits,score=iterate_find_inliers(chits,lhits)
    for i in chits.index:
        hits.loc[i,track]

In [ ]:
hits.query('track_id==67941')

In [ ]:
len(restricted_labels)

In [ ]:
cum_err=np.sort(err_tot) + 50/np.array(range(1,len(err_tot)+1))
max_idx=np.argmin(cum_err)
inliers=lhits.sort_values('err').index[:max_idx+1]
cum_err

In [ ]:
hits.sample(n=10)

In [ ]:
hits.query('track_id==67045').sort_values('rho')

In [ ]:
hits.query('particle_id==180145015886970880').sort_values('rho')

In [ ]:
part_index=[]
for hit_id in hits.query('particle_id==180145015886970880')['hit_id'].values:
    part_index.extend(np.where(hits['hit_id'].values==hit_id)[0].tolist())
part_index

In [ ]:
[decode_pair(x) for x in hit_pairs[15071]]

In [ ]:
for test_pair in hit_pairs[19501]:
    p1,p2=decode_pair(test_pair)
    if p1 in part_index and p2 in part_index:
        print('test pair: {}'.format((p1,p2)))
        for nbr,nscore in graph[test_pair]:
            n1,n2=decode_pair(nbr)
            if n1 in part_index and n2 in part_index:
                print('{}: {} [{}]'.format((n1,n2),nscore,pair_labels[nbr] if nbr in pair_labels else 0))
                for nnbr,nnscore in graph[nbr]:
                    nn1,nn2=decode_pair(nnbr)
                    if nn1 in part_index and nn2 in part_index:
                        print('-{}: {} [{}]'.format((nn1,nn2),nnscore,pair_labels[nnbr] if nnbr in pair_labels else 0))

In [ ]:
l,c=np.unique([pair_labels[p] for p in hit_pairs[15071] if pair_labels[p] > 0],return_counts=True)

In [ ]:
[(a,b) for a,b in zip(l,c) if b>1]

In [ ]:
all_pairs_list=list(all_pairs)
pair_label_list=[pair_labels[p] for p in all_pairs_list]

In [ ]:
ls,cs=np.unique(pair_label_list,return_counts=True)
order=np.argsort(cs)[::-1]
list(zip(ls[order],cs[order]))

In [ ]:
c=np.where(np.array(pair_label_list,dtype=np.int64)==67045)
hits.loc[np.unique([decode_pair(all_pairs_list[x]) for x in c[0]]).flatten()].sort_values('rho')

In [ ]:
selected_pairs=np.array(all_pairs_list)[np.array(pair_label_list)==8990]
spairs=set()
for pair in selected_pairs:
    a,b=decode_pair(pair)
    spairs.add(a)
    spairs.add(b)

In [ ]:
len(selected_pairs)

In [ ]:
filt_hits=hits.loc[list(spairs)].sort_values('rho')

In [ ]:
def angular_distance(theta1, theta2):
    return np.abs(((theta1-theta2+np.pi) % (2*np.pi)) - np.pi)
def eta(th):
    return -np.log(np.tan(th/2.0))
def pairwise_angular(x,xp,y,yp,rref,eps=1e-9):
    a=yp-y
    b=xp-x
    c=xp*y-yp*x
    inv=np.sign(c)/np.sqrt(a*a+b*b+eps)
    fd=c*inv
    d=np.sqrt(x*x+y*y)
    theta0=np.arctan2(y,x)
    thetaa=np.arctan2(a,b)
    add_pi = (angular_distance(theta0,thetaa+np.pi) < angular_distance(theta0,thetaa)).astype(np.float32)
    thetaa=thetaa + add_pi * np.pi
    thetai = np.arcsin((d/rref)*np.sin(theta0-thetaa)) + thetaa
    return fd.flatten(),thetaa.flatten(),thetai.flatten()

In [ ]:
idx1=[]
idx2=[]
for pair in selected_pairs:
    a,b=decode_pair(pair)
    idx1.append(a)
    idx2.append(b)

In [ ]:
lh,lc=np.unique(np.array(idx1+idx2),return_counts=True)
chits=hits.loc[lh[lc>6]]
lhits=hits.loc[lh]
chits

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(lhits['rho'].values,lhits['x'].values)
plt.scatter(chits['rho'].values,chits['x'].values)

In [ ]:
plt.scatter(lhits['rho'].values,lhits['y'].values)
plt.scatter(chits['rho'].values,chits['y'].values)

In [ ]:
plt.scatter(lhits['rho'].values,lhits['z'].values)
plt.scatter(chits['rho'].values,chits['z'].values)

In [ ]:
deg=2
fx=np.poly1d(np.polyfit(chits['rho'].values,chits['x'].values,deg))
fy=np.poly1d(np.polyfit(chits['rho'].values,chits['y'].values,deg))
fz=np.poly1d(np.polyfit(chits['rho'].values,chits['z'].values,deg))

In [ ]:
rho=lhits['rho'].values
xerr=(lhits['x'].values-fx(rho))
yerr=(lhits['y'].values-fy(rho))
zerr=(lhits['z'].values-fz(rho))
err_tot=np.log(1+(xerr*xerr+yerr*yerr+zerr*zerr)/np.power(lhits['sigma'].values,2))/np.log(2)
lhits['err']=err_tot
sorted_lhits=lhits.sort_values('err')
used_modules=set()
for i,ids in zip(sorted_lhits.index,sorted_lhits[['volume_id','layer_id','module_id']].values):
    a,b,c=ids
    if (a,b,c) in used_modules:
        lhits.loc[i,'err']=lhits.loc[i,'err']+10
    used_modules.add((a,b,c))

In [ ]:
lhits.sort_values('err')

In [ ]:
k=3
cum_err=np.sqrt(np.cumsum(np.sort(err_tot))[k:])/np.array(range(1,len(err_tot)+1-k))
max_idx=np.argmin(cum_err)+k
inliers=lhits.sort_values('err').index[:max_idx+1]
cum_err[:10]

In [ ]:
cum_err=np.sort(err_tot) + 50/np.array(range(1,len(err_tot)+1))
max_idx=np.argmin(cum_err)
inliers=lhits.sort_values('err').index[:max_idx+1]
cum_err

In [ ]:
chits=lhits.loc[inliers]
chits

In [ ]:
r=filt_hits['r'].values
z=filt_hits['z'].values
df1 = hits.take(idx1)
df2 = hits.take(idx2)
pairs = pd.DataFrame()
pairs['idx1']=df1.index
pairs['idx2']=df2.index
reference_sphere=500.0
pairs['rz_d'],pairs['rz_tha'],pairs['rz_thi'] = pairwise_angular(df1['z'].values, df2['z'].values, df1['r'].values, df2['r'].values, reference_sphere)
rref = 1.0 / reference_sphere / np.sin(pairs['rz_thi'].values)
pairs['uv_d'],pairs['uv_tha'],pairs['uv_thi'] = pairwise_angular(df1['u'].values, df2['u'].values, df1['v'].values, df2['v'].values, rref)

In [ ]:
pairs=pairs.dropna()
pairs['rz_cosa']=np.cos(pairs['rz_tha'].values)
pairs['rz_sina']=np.sin(pairs['rz_tha'].values)
pairs['rz_cosi']=np.cos(pairs['rz_thi'].values)
pairs['rz_sini']=np.sin(pairs['rz_thi'].values)
pairs['uv_cosa']=np.cos(pairs['uv_tha'].values)
pairs['uv_sina']=np.sin(pairs['uv_tha'].values)
pairs['uv_cosi']=np.cos(pairs['uv_thi'].values)
pairs['uv_sini']=np.sin(pairs['uv_thi'].values)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(pairs['rz_sin_thi'].values,pairs['rz_sin_tha'])

In [ ]:
pairs.dropna(inplace=True)

In [ ]:
import hdbscan
cl=hdbscan.HDBSCAN(min_cluster_size=10)
X=pairs[['rz_cosa', 'rz_sina', 'rz_cosi', 'rz_sini', 'uv_cosa', 'uv_sina', 'uv_cosi', 'uv_sini']].values
filt_labels=cl.fit_predict(X)

In [ ]:
np.unique(filt_labels,return_counts=True)

In [ ]:
pairs['label']=filt_labels

In [ ]:
pairs['good']=hits.loc[pairs['idx1'].values]['particle_id'].values==hits.loc[pairs['idx2'].values]['particle_id'].values

In [ ]:
pairs[['label','good']].query('good')

In [ ]:
pairs[['label','good']].query('label==2')